In [4]:
import os
import nbformat
import pkg_resources

def extract_imports_from_notebook(notebook_path, custom_libraries):
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)
    
    imports = set()
    for cell in notebook.cells:
        if cell.cell_type == 'code':
            code = cell.source
            lines = code.split('\n')
            for line in lines:
                line = line.strip()
                if line.startswith('import ') or line.startswith('from '):
                    parts = line.replace(',', ' ').split()
                    if parts[0] == 'import':
                        for part in parts[1:]:
                            lib = part.split('.')[0]
                            if lib and lib not in custom_libraries:
                                imports.add(lib)
                    elif parts[0] == 'from':
                        lib = parts[1].split('.')[0]
                        if lib and lib not in custom_libraries:
                            imports.add(lib)
    return imports

def get_installed_version(lib_name):
    try:
        version = pkg_resources.get_distribution(lib_name).version
    except pkg_resources.DistributionNotFound:
        version = None
    return version

def generate_requirements(custom_libraries=None):
    if custom_libraries is None:
        custom_libraries = []
    
    current_directory = os.getcwd()
    all_imports = set()
    
    for root, _, files in os.walk(current_directory):
        for file in files:
            if file.endswith('.ipynb'):
                notebook_path = os.path.join(root, file)
                imports = extract_imports_from_notebook(notebook_path, custom_libraries)
                all_imports.update(imports)
    
    requirements = []
    for lib in sorted(all_imports):
        version = get_installed_version(lib)
        if version:
            requirements.append(f'{lib}=={version}')
    
    with open('requirements.txt', 'w') as f:
        for req in requirements:
            f.write(req + '\n')

    print(f"requirements.txt generated with {len(requirements)} libraries.")

# List your custom libraries here
custom_libraries = ['model', 'your_custom_library']
generate_requirements(custom_libraries)


requirements.txt generated with 24 libraries.
